### 유튜브 댓글 크롤링 & 분석(검색어: 타다 금지법)

In [1]:
import requests
import time
import scrapy

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from scrapy.http import TextResponse
from bs4 import BeautifulSoup

#### 1) 유튜브 첫 화면에서 '타다금지법' 검색하여, 이번주 업로드된 동영상 제목, url 가져오기
####   (as of 3/15)

In [2]:
options = Options()
options.headless = True
driver = webdriver.Chrome('/home/ubuntu/chromedriver', options=options)
driver.get('https://www.youtube.com/results?search_query=타다 금지법&sp=EgIIAw%253D%253D')

page=driver.page_source
soup = BeautifulSoup(page,'lxml')

all_ = soup.find_all('a','yt-simple-endpoint style-scope ytd-video-renderer')

#제목
titles = [all_[n].text.strip() for n in range(0,len(all_))]
#주소 
links= ["https://www.youtube.com/" + all_[n].get('href') for n in range(0,len(all_))]

titles, links

(["'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",
  '\'타다 금지법\' 국회 통과…사측 "한 달 내 서비스 중단" / JTBC 뉴스룸',
  '타다, 법안 공포 후 중단…"희망고문 못 견뎌" / 연합뉴스TV (YonhapnewsTV)',
  '타다, 파견 직원들 권고사직 요구…감원 착수 / 연합뉴스TV (YonhapnewsTV)',
  '타다 끝낼까 변화할까?…1년 6개월 후속조치 관건 / KBS뉴스(News)',
  '[톡톡] 타다금지법 국회통과 - 3/9(월) 톡톡! 뉴스와 상식/ YTN 라디오',
  '[KBS열린토론] 타다 금지법 이후, 차량 공유서비스는 어디로? / 정준희, 구태언, 권용주, 최성진(200310)',
  "[보이는라디오] '타다금지법' 국회통과, 혁신의 한계인가 상생인가 3/10(화) 뉴스FM, 조현지입니다 YTN RADIO",
  '타다, 국회에 제동 걸려…대통령 거부권 행사 촉구 / 연합뉴스TV (YonhapnewsTV)',
  "[극한상사] '타다금지법' 국회통과, 혁신의 한계인가 상생인가 3/10(화) 뉴스FM, 조현지.../ YTN 라디오",
  '[자막뉴스] 타다 끝낼까 변화할까?…1년 6개월 후속조치 관건 / KBS뉴스(News)',
  "타다, 4월 10일까지만 운행…드라이버 1만여 명 '생계 막막' / JTBC 아침&",
  '시동꺼진 ‘타다’...남은 건 대통령의 선택뿐?',
  "'타다 베이직' 4월 11일 중단…1만명 넘는 드라이버 어쩌나 / JTBC 뉴스룸",
  "타다, 결국엔 못 타다…드라이버는 '생계 막막', 승객은 '아쉬움'",
  '"변호사가 보는 타다금지법" 타다는 혁신이다? VS 꼼수다?',
  '타다 타다금지법 통과 이후... 청와대 호소??? 04.11 서비스 종료???',
  '타다 금지법 통과 해석: 4. 타다의 대응과 차량공유 산업의 전략은?',
  '국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3',
  '[뉴스속보] \'타다금지법\' 국회 통과

#### 2) 동영상 상세 링크 접속하여 댓글 남긴 사람 id, comment 가져오기(우선 sample로 3개)

In [3]:
comment_data = pd.DataFrame({'제목': [],
                             'user_id': [],
                             '댓글': [],
                             '좋아요 수': []
                             })

for i in range(0, 3):
    start_url = links[i]
    #driver = webdriver.Chrome('/home/ubuntu/chromedriver', options=options)
    driver.get(start_url)
    
    body = driver.find_element_by_tag_name("body")

    num_of_pagedowns = 10
    
    while num_of_pagedowns: 
        body.send_keys(Keys.PAGE_DOWN) 
        time.sleep(2)
        num_of_pagedowns -= 1 
    
    try: 
        driver.find_element_by_xpath('//*[@id="sort-menu"]').click() 
        driver.find_element_by_xpath('//*[@id="menu"]/a[2]/paper-item/paper-item-body/div[text()="최근 날짜순"]').click() 
    
    except Exception as e: 
        pass
    #제목
    title = driver.find_element_by_css_selector('#container > h1 > yt-formatted-string').text
    #사용자 id    
    user_ids = driver.find_elements_by_css_selector('#author-text > span')
    # 댓글
    comments = driver.find_elements_by_css_selector('#content-text')
    #좋아요 수
    like_nums = driver.find_elements_by_css_selector('#vote-count-middle')
    
    datas = []

    for j in range(len(comments)):
        title = title
        comment = comments[j].text
        user_id = user_ids[j].text
        like_num = like_nums[j].text
        
        datas.append({
        "제목" : title,
        "user_id" : user_id,
        "댓글" : comment,
        "좋아요 수" : like_num
         })

    result_data=pd.DataFrame(datas)
    result = result_data[["제목", "user_id", "댓글", "좋아요 수"]]
    comment_data = comment_data.append(result, ignore_index=True)

In [4]:
comment_data 

,제목,user_id,댓글,좋아요 수
0,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",윤석,사기꾼 이재웅 투자 이외는 돈이 없어 1년6개월 못버티고 한달도 안되서 운행정...,1
1,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",c초코,4등!!!,
2,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",남짱,얼론이타다를왜자세히다루는지이해가안됨,
3,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",개동이,4차산업 공유경제 뭘 공유했는지 아는사람? 이재웅 서비스 접는다며...,
4,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",포푸라,부산만 하더라도 택시가 24000대인데\n무분별하게 타다같은 업체가 렌탈카니발 증차...,
5,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",최규선,당연하다 지금도 택시가 맗은데 교통 만혼잡하다,2
6,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",kms8964,대통령이 니 형이냐! ! !,1
7,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",김종한,타다가개색키들의집단이지뭥ㅑ,3
8,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",체스,성폭행은 죄다.\n택시기사든데~~,
9,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",R몽실통통,택시업계에서 성실히 법을 준수하며 생계를 유지해온 택시업계 종사자를 보호하고 타다를...,1


In [6]:
### 데이터프레임 엑셀로 만들기
comment_data.to_csv('comment_data.csv')

In [7]:
df = pd.read_csv('comment_data.csv')
df.head()

,Unnamed: 0,제목,user_id,댓글,좋아요 수
0,0,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",윤석,사기꾼 이재웅 투자 이외는 돈이 없어 1년6개월 못버티고 한달도 안되서 운행정...,1.0
1,1,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",c초코,4등!!!,NaN
2,2,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",남짱,얼론이타다를왜자세히다루는지이해가안됨,NaN
3,3,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",개동이,4차산업 공유경제 뭘 공유했는지 아는사람? 이재웅 서비스 접는다며...,NaN
4,4,"'타다 금지법' 국회 통과…타다 측, 문 대통령에 '거부권 행사' 요청",포푸라,부산만 하더라도 택시가 24000대인데\n무분별하게 타다같은 업체가 렌탈카니발 증차...,NaN
